In [1]:
import pickle
from utils.visualize import visualize_mol
from rdkit import Chem

In [2]:
load_path = "reproduce/wb97xd3/train_from_scratch/samples_all.pkl"
with open(load_path, "rb") as f:
    samples = pickle.load(f)

In [13]:
visualize_mol(samples[0]['rdmol'][0])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
visualize_mol(samples[0]['rdmol'][1])

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [18]:
samples[0]['pos']

tensor([[ 1.7909,  0.5970,  0.5856],
        [ 0.5308,  0.0967,  0.9509],
        [-0.0460, -0.6950, -0.0457],
        [-1.4518, -0.6923, -0.0935],
        [-1.5587,  0.3914, -1.6083],
        [ 2.1620,  1.1879,  1.4224],
        [ 2.5022, -0.2135,  0.3815],
        [ 1.7264,  1.2296, -0.3090],
        [-0.4254,  0.0571, -1.2036],
        [ 0.4812, -1.6246, -0.2605],
        [-1.9702, -1.5311, -0.5363],
        [-2.0237, -0.0912,  0.6038],
        [-1.7790,  1.3075, -1.3976]])

In [19]:
samples[0]['pos_gen']

tensor([[ 2.0798,  0.2718,  0.3586],
        [ 0.9146, -0.3384, -0.1378],
        [-0.2350,  0.4564, -0.0229],
        [-1.3323,  0.0328, -0.8303],
        [-2.1698, -0.9367,  0.4691],
        [ 2.5712,  0.8044, -0.4498],
        [ 1.8907,  0.9368,  1.1954],
        [ 2.7501, -0.5205,  0.7081],
        [-1.1485, -0.3253,  0.8040],
        [-0.1172,  1.4960,  0.1942],
        [-1.1780, -0.7417, -1.5747],
        [-2.1109,  0.7248, -1.0811],
        [-1.9145, -1.8603,  0.3673]])

In [21]:
samples[0]['atom_type']

tensor([6, 8, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1])

In [27]:
samples[0]['smiles']

'[C:1]([O:2][C:3]([C:4]([O:5][H:13])([H:11])[H:12])([H:9])[H:10])([H:6])([H:7])[H:8]>>[C:1]([O:2][C:3](=[C:4]([H:11])[H:12])[H:10])([H:6])([H:7])[H:8].[O:5]([H:9])[H:13]'

In [39]:
visualize_mol(Chem.MolFromSmarts('[C:1]([O:2][C:3]([C:4]([O:5][H:13])([H:11])[H:12])([H:9])[H:10])([H:6])([H:7])[H:8]'))

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [40]:
visualize_mol(Chem.MolFromSmarts('[C:1]([O:2][C:3](=[C:4]([H:11])[H:12])[H:10])([H:6])([H:7])[H:8].[O:5]([H:9])[H:13]'))

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [16]:
r = Chem.MolFromSmarts('[C:1]([O:2][C:3]([C:4]([O:5][H:13])([H:11])[H:12])([H:9])[H:10])([H:6])([H:7])[H:8]')
p = Chem.MolFromSmarts('[C:1]([O:2][C:3](=[C:4]([H:11])[H:12])[H:10])([H:6])([H:7])[H:8].[O:5]([H:9])[H:13]')

In [17]:
r.GetConformer(0)

ValueError: Bad Conformer Id

In [42]:
r.GetNumAtoms(), p.GetNumAtoms()

(13, 13)

In [35]:
samples[0]['rdmol'][0].GetNumAtoms(), samples[0]['rdmol'][1].GetNumAtoms()

(13, 13)

In [52]:
[a.GetAtomMapNum() for a in r.GetAtoms()]

[1, 2, 3, 4, 5, 13, 11, 12, 9, 10, 6, 7, 8]

In [53]:
samples[0]['atom_type']

tensor([6, 8, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1])

In [54]:
[a.GetAtomicNum() for a in r.GetAtoms()]

[6, 8, 6, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1]

In [55]:
[a.GetSymbol() for a in r.GetAtoms()]

['C', 'O', 'C', 'C', 'O', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']

In [20]:
def pos2xyz(ind, r, pos):
    atom_type = [a.GetSymbol() for a in r.GetAtoms()]
    N = len(atom_type)
    description = f'generated ts {ind:0>6d}'
    pos_lines = []
    for t, p in zip(atom_type, pos):
        pos_lines.append('\t'.join([t]+list(map(str, p.tolist()))))
    return '\n'.join([str(N), description, *pos_lines])

In [42]:
from tqdm import tqdm

import os
import shutil
import subprocess
import multiprocessing as mp
import numpy as np
from easydict import EasyDict
import pandas as pd

def check_and_clear_directory(directory_path):
    if os.path.exists(directory_path):
        print(f"Folder {directory_path} exists. Empting...")
        for item in os.listdir(directory_path):
            item_path = os.path.join(directory_path, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
        print(f"Folder {directory_path} has been emptied.")
    else:
        print(f"Folder {directory_path} does not exist. Creating...")
        os.makedirs(directory_path)
        print(f"Folder {directory_path} has been created.")

def calulate_dmse(sample, ind):
    if '.' in sample['smiles']:
        return None
    r = sample['rdmol'][0]
    # reference ts
    ref_ts = pos2xyz(ind, r, sample['pos'])
    # generated ts
    gen_ts = pos2xyz(ind, r, sample['pos_gen'])
    with open(f'{path}/ref_samples_{ind:0>6d}.xyz', 'w') as f:
        f.write(ref_ts)
    with open(f'{path}/gen_samples_{ind:0>6d}.xyz', 'w') as f:
        f.write(gen_ts)
    result = subprocess.run(['python',
                             './utils/evaluation/calculate_rmsd.py', 
                             '--no-hydrogen',
                             '--dmae',
                             f'{path}/ref_samples_{ind:0>6d}.xyz',
                             f'{path}/gen_samples_{ind:0>6d}.xyz'
                             ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print("Return code: ", result.returncode)
        print("Output: ", result.stdout)
        print("Error: ", result.stderr)
        raise ValueError("calculate_rmsd failed.")
    
    return float(result.stdout)

def print_covmat_results(results, print_fn=print):
    df = pd.DataFrame(
        {
            "COV_mean": np.mean(results.Coverage, 0),
            "COV_median": np.median(results.Coverage, 0),
            "COV_std": np.std(results.Coverage, 0),
        },
        index=results.thresholds,
    )
    print_fn("\n" + str(df))
    print_fn(
        "MAT_mean: %.4f | MAT_median: %.4f | MAT_std %.4f"
        % (
            np.mean(results.Matching),
            np.median(results.Matching),
            np.std(results.Matching),
        )
    )
    return df

# load_path = "reproduce/wb97xd3/train_from_scratch/samples_all.pkl"
load_path = "reproduce/wb97xd3/samples_all.pkl"
with open(load_path, "rb") as f:
    samples = pickle.load(f)

# path = 'reproduce/wb97xd3/train_from_scratch/xyz_files'
path = 'reproduce/wb97xd3/xyz_files'
check_and_clear_directory(path)

pool = mp.Pool(32)

cov01s = []
cov02s = []
cov05s = []
cov06s = []
cov07s = []
cov08s = []
cov10s = []
cov20s = []
cov30s = []
mats = []

for d_mae in tqdm(
    pool.starmap(calulate_dmse, zip(samples, range(len(samples)))), total=len(samples)):
    if d_mae is None:
        continue
    cov01s.append(1 if d_mae < 0.1 else 0)
    cov02s.append(1 if d_mae < 0.2 else 0)
    cov05s.append(1 if d_mae < 0.5 else 0)
    cov06s.append(1 if d_mae < 0.6 else 0)
    cov07s.append(1 if d_mae < 0.7 else 0)
    cov08s.append(1 if d_mae < 0.8 else 0)
    cov10s.append(1 if d_mae < 1.0 else 0)
    cov20s.append(1 if d_mae < 2.0 else 0)
    cov30s.append(1 if d_mae < 3.0 else 0)
    mats.append(d_mae)

cov_scores = np.vstack([cov01s, cov02s, cov05s, cov06s, cov07s, cov08s, cov10s, cov20s, cov30s]).T
mat_scores = np.array(mats)

results = EasyDict(
    {
        "Coverage": cov_scores,
        "Matching": mat_scores,
        "thresholds": [0.1, 0.2, 0.5, 0.6, 0.7, 0.8, 1.0, 2.0, 3.0],
    }
)

results_df = print_covmat_results(results)
    

Folder reproduce/wb97xd3/train_from_scratch/xyz_files exists. Empting...
Folder reproduce/wb97xd3/train_from_scratch/xyz_files has been emptied.


100%|██████████| 2394/2394 [00:00<00:00, 285983.42it/s]


     COV_mean  COV_median   COV_std
0.1  0.437796         0.0  0.496116
0.2  0.736374         1.0  0.440599
0.5  0.989336         1.0  0.102712
0.6  0.997038         1.0  0.054344
0.7  0.999408         1.0  0.024332
0.8  1.000000         1.0  0.000000
1.0  1.000000         1.0  0.000000
2.0  1.000000         1.0  0.000000
3.0  1.000000         1.0  0.000000
MAT_mean: 0.1430 | MAT_median: 0.1148 | MAT_std 0.1103


In [ ]:
from tqdm import tqdm
outputs = []
for ind, sample in tqdm(enumerate(samples)):
    r = sample['rdmol'][0]
    # reference ts
    ref_ts = pos2xyz(ind, r, sample['pos'])
    # generated ts
    gen_ts = pos2xyz(ind, r, sample['pos_gen'])
    outputs.extend([ref_ts, gen_ts])
with open('reproduce/wb97xd3/train_from_scratch/samples_all.xyz', 'w') as f:
    f.write('\n'.join(outputs))

In [2]:
import ase
from clustering import convert_from_atoms
from copy import deepcopy
def align(atom_type, pos, pos_gen, smiles):
    ref_ts = ase.Atoms(positions=pos.numpy(), numbers=atom_type.numpy())
    gen_ts = ase.Atoms(positions=pos_gen.numpy(), numbers=atom_type.numpy())
    smarts = deepcopy(smiles)
    gen_ts = convert_from_atoms(gen_ts, [ref_ts], smarts)
    return gen_ts[0].positions

In [2]:
import pickle
from utils.visualize import visualize_mol
from rdkit import Chem
import numpy as np

# load_path = "reproduce/wb97xd3/train_from_scratch/samples_all.pkl"
load_path = "reproduce/wb97xd3/samples_all.pkl"
with open(load_path, "rb") as f:
    samples = pickle.load(f)

from copy import deepcopy
from rdkit.Chem import rdDetermineBonds

data_list = []
for sample in samples:
    sample = deepcopy(sample)
    sample['pos_ref'] = sample['pos']
    data_list.append(sample)

from utils.evaluation.covmat import CovMatEvaluator, print_covmat_results

eval = CovMatEvaluator(ratio=1, num_workers=8, thresholds=np.array([0.1, 0.2]), use_force_field=False)
rst = eval(data_list)
print_covmat_results(rst)

smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles abnormal
smiles a

  2%|▏         | 31/1688 [00:00<00:22, 74.55it/s]

Python argument types in
    Mol.__init__(Mol, NoneType)
did not match C++ signature:
    __init__(_object*, RDKit::ROMol mol, bool quickCopy=False, int confId=-1)
    __init__(_object*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > pklString, unsigned int propertyFlags)
    __init__(_object*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > pklString)
    __init__(_object*)


 41%|████      | 690/1688 [00:08<00:11, 83.70it/s] 

Python argument types in
    Mol.__init__(Mol, NoneType)
did not match C++ signature:
    __init__(_object*, RDKit::ROMol mol, bool quickCopy=False, int confId=-1)
    __init__(_object*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > pklString, unsigned int propertyFlags)
    __init__(_object*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > pklString)
    __init__(_object*)


100%|██████████| 1688/1688 [00:19<00:00, 85.24it/s] 


     COV-R_mean  COV-R_median  COV-R_std  COV-P_mean  COV-P_median  COV-P_std
0.1    0.060491           0.0   0.238395    0.060491           0.0   0.238395
0.2    0.136106           0.0   0.342901    0.136106           0.0   0.342901
MAT-R_mean: 0.7208 | MAT-R_median: 0.7270 | MAT-R_std 0.3994
MAT-P_mean: 0.7208 | MAT-P_median: 0.7270 | MAT-P_std 0.3994


,COV-R_mean,COV-R_median,COV-R_std,COV-P_mean,COV-P_median,COV-P_std
0.1,0.060491,0.0,0.238395,0.060491,0.0,0.238395
0.2,0.136106,0.0,0.342901,0.136106,0.0,0.342901


In [8]:
feat_dict = pickle.load(open('data/TS/wb97xd3/feat_dict.pkl', "rb"))

In [9]:
feat_dict

{'GetIsAromatic': {False: 0, True: 1},
 'GetFormalCharge': {0: 0, -1: 1, 1: 2},
 'GetHybridization': {rdkit.Chem.rdchem.HybridizationType(4): 0,
  rdkit.Chem.rdchem.HybridizationType(3): 1,
  rdkit.Chem.rdchem.HybridizationType(1): 2,
  rdkit.Chem.rdchem.HybridizationType(2): 3},
 'GetTotalNumHs': {0: 0, 2: 1, 1: 2},
 'GetTotalValence': {4: 0, 3: 1, 2: 2, 1: 3},
 'GetTotalDegree': {4: 0, 3: 1, 2: 2, 1: 3},
 'GetChiralTag': {rdkit.Chem.rdchem.ChiralType(0): 0,
  rdkit.Chem.rdchem.ChiralType(1): 1,
  rdkit.Chem.rdchem.ChiralType(2): 2},
 'IsInRing': {False: 0, True: 1}}

In [19]:
import pickle
from utils.visualize import visualize_mol
from rdkit import Chem
import numpy as np
import rmsd
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from tqdm import tqdm

# load_path = "reproduce/wb97xd3/train_from_scratch/samples_all.pkl"
load_path = "reproduce/wb97xd3/samples_all.pkl"
with open(load_path, "rb") as f:
    samples = pickle.load(f)


def dmae(A, B):
    pdist_A = pdist(A)
    pdist_B = pdist(B)
    d_A = squareform(pdist_A)
    d_B = squareform(pdist_B)
    assert d_A.shape == d_B.shape
    
    mae = 0
    for i in range(d_A.shape[0]):
        for j in range(d_A.shape[1]):
            if i < j:
                mae += abs(d_A[i, j] - d_B[i, j])

    return (mae * 2) / (d_A.shape[0] * (d_A.shape[0] - 1))
                
cov01 = 0
cov02 = 0
mat = 0

for sample in tqdm(samples):
    gen_ts = sample['pos_gen']
    ref_ts = sample['pos']
    U = rmsd.kabsch(gen_ts, ref_ts)
    gen_ts_aligned = np.dot(gen_ts, U)
    d_mae = dmae(gen_ts_aligned, ref_ts)
    if d_mae < 0.1:
        cov01 += 1
    elif d_mae < 0.2:
        cov02 += 1
    mat += d_mae

print(f'cov01: {cov01 / len(samples):.4f}')
print(f'cov02: {cov02 / len(samples):.4f}')
print(f'mat: {mat / len(samples):.4f}')

100%|██████████| 2394/2394 [00:00<00:00, 5566.14it/s]

cov01: 0.1696
cov02: 0.2774
mat: 0.2256


In [12]:
ref_ts_1

array([[ 1.9098341e+00,  2.3043552e-01,  4.5406687e-01],
       [ 9.0976965e-01,  2.9535654e-01, -5.2954531e-01],
       [-3.4679922e-01,  6.0580426e-01, -2.8439537e-03],
       [-1.4499933e+00,  7.8193374e-02, -6.9800532e-01],
       [-1.8160660e+00, -1.2463990e+00,  5.6373388e-01],
       [ 2.8471050e+00, -4.5801103e-03, -4.9277306e-02],
       [ 2.0204890e+00,  1.1875715e+00,  9.7955102e-01],
       [ 1.6894627e+00, -5.4558343e-01,  1.1982555e+00],
       [-8.8730586e-01, -4.4658592e-01,  8.0380827e-01],
       [-4.3245295e-01,  1.6015035e+00,  4.3231905e-01],
       [-2.4125667e+00,  5.6258798e-01, -6.1365163e-01],
       [-1.3013266e+00, -5.3969151e-01, -1.5759820e+00],
       [-1.5019572e+00, -2.1180584e+00,  2.9235250e-01]], dtype=float32)

In [13]:
temp = squareform(pdist(ref_ts_1))

In [14]:
temp

array([[0.        , 1.40422101, 2.33282305, 3.5551214 , 4.00941383,
        1.08952486, 1.09749142, 1.09753623, 2.89908076, 2.71414981,
        4.46469448, 3.87630591, 4.145105  ],
       [1.40422101, 0.        , 1.39741098, 2.37571454, 3.31699409,
        2.0183874 , 2.07535942, 2.07373985, 2.35749672, 2.10541312,
        3.33412734, 2.58481754, 3.50948917],
       [2.33282305, 1.39741098, 0.        , 1.40664837, 2.43113399,
        3.25203775, 2.62823257, 2.62958097, 1.43190794, 1.09000933,
        2.15461105, 2.16749772, 2.973375  ],
       [3.5551214 , 2.37571454, 1.40664837, 0.        , 1.86562058,
        4.34657947, 4.01112977, 3.72036103, 1.68744021, 2.15255549,
        1.08087985, 1.08384801, 2.40977818],
       [4.00941383, 3.31699409, 2.43113399, 1.86562058, 0.        ,
        4.86446927, 4.56248517, 3.63077012, 1.24897246, 3.16894365,
        2.23930429, 2.31144459, 0.96545447],
       [1.08952486, 2.0183874 , 3.25203775, 4.34657947, 4.86446927,
        0.        , 1.77848

In [15]:
temp.shape

(13, 13)